# Optimize table data layout by partitioning and clustering using compaction
<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

Through compaction, whether manual or running automatically, you can change the number and size of segments that make up a table.

This tutorial demonstrates how to work with [compaction](https://druid.apache.org/docs/latest/data-management/compaction) to partition and cluster the segments for an existing table.

In this tutorial you perform the following tasks:

- Create a table using batch ingestion with a very high number of segments.
- Re-partition your data using a compaction job, reducing the number of segments by increasing their size.
- Re-cluster your data by changing the secondary partitioning scheme of an existing table.

## Prerequisites

This tutorial works with Druid 29.0.0 or later.

Launch this tutorial and all prerequisites using the `druid-jupyter` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).

## Initialization

The following cells set up the notebook and learning environment ready for use.

### Set up a connection to Apache Druid

Run the next cell to set up the Druid Python client's connection to Apache Druid.

If successful, the Druid version number will be shown in the output.

In [ ]:
import druidapi
import os

druid_headers = {'Content-Type': 'application/json'}

if 'DRUID_HOST' not in os.environ.keys():
    druid_host=f"http://localhost:8888"
else:
    druid_host=f"http://{os.environ['DRUID_HOST']}:8888"

print(f"Opening a connection to {druid_host}.")
druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql
status_client = druid.status

status_client.version

### Import additional modules

Run the following cell to import additional Python modules that you will use to call Druid APIs directly.

In [ ]:
import requests
import json

## Create a table using batch ingestion

Run the following cell to create a table using batch ingestion. Notice that this ingestion will partition the incoming data by hour.

When completed, you'll see a description of the final table.

In [ ]:
table_name = 'example-wikipedia-compaction'

sql='''
REPLACE INTO "''' + table_name + '''" OVERWRITE ALL
WITH "ext" AS (
  SELECT *
  FROM TABLE(
    EXTERN(
      '{"type":"http","uris":["https://druid.apache.org/data/wikipedia.json.gz"]}',
      '{"type":"json"}'
    )
  ) EXTEND ("isRobot" VARCHAR, "channel" VARCHAR, "timestamp" VARCHAR, "flags" VARCHAR, "isUnpatrolled" VARCHAR, "page" VARCHAR, "diffUrl" VARCHAR, "added" BIGINT, "comment" VARCHAR, "commentLength" BIGINT, "isNew" VARCHAR, "isMinor" VARCHAR, "delta" BIGINT, "isAnonymous" VARCHAR, "user" VARCHAR, "deltaBucket" BIGINT, "deleted" BIGINT, "namespace" VARCHAR, "cityName" VARCHAR, "countryName" VARCHAR, "regionIsoCode" VARCHAR, "metroCode" BIGINT, "countryIsoCode" VARCHAR, "regionName" VARCHAR)
)
SELECT
  TIME_PARSE("timestamp") AS "__time",
  "namespace",
  "page",
  "user",
  "channel",
  "added",
  "deleted",
  "commentLength",
  "isRobot",
  "isAnonymous",
  "regionIsoCode",
  "countryIsoCode"
FROM "ext"
PARTITIONED BY HOUR
'''

display.run_task(sql)
sql_client.wait_until_ready(f'{table_name}')
display.table(f'{table_name}')

## View the layout of a table

Use Druid's `SYS.SEGMENTS` table to get information about a TABLE's segments. Run the cell below to see the segments created by the ingestion above.

In [ ]:
sql=f'''
SELECT
  "start",
  "end",
  "size"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

Since you used PARTITIONED BY HOUR, you will see one segment per hour for the entire ingested data set.

## Apply changes to data layout through compaction

Compaction is a special type of native [Druid task](https://druid.apache.org/docs/latest/ingestion/tasks#all-task-types) that, like streaming ingestion, uses JSON specifications to define behaviors. Each contains:

* An [ioConfig](https://druid.apache.org/docs/latest/data-management/manual-compaction#compaction-io-configuration), defining the connection to the source data.
* A [tuningConfig](https://druid.apache.org/docs/latest/ingestion/native-batch#tuningconfig), detailing specific controls.
* What happens to the incoming data (as you would find in a [`dataSchema`](https://druid.apache.org/docs/latest/ingestion/ingestion-spec#dataschema) in streaming ingestion).
  * The dimensions to put into the resulting data given in a `dimensionsSpec`.
  * Any filters or calculations to do on the data as listed in the `transformsSpec`.
  * Any aggregation that should be done, as given in the `metricsSpec` when `rollup` is enabled.
 
In the cells that follow you will see various examples of how to use compaction to effect segment layout.

### Apply a different PARTITION BY scheme

Rows per segment being very small is [one reason](https://druid.apache.org/docs/latest/data-management/compaction#compaction-guidelines) to run a compaction job to change the partitioning scheme.

To affect the PARTITION BY scheme in compaction you will use a [`granularitySpec`](https://druid.apache.org/docs/latest/data-management/manual-compaction/#compaction-granularity-spec) and a daily primary partitioning scheme by setting the `segmentGranularity` to DAY.

Run the next cell to build up a JSON ingestion specification for a compaction job:

In [ ]:
compaction_ioConfig_inputSpec = {
    "type" : "interval",
    "interval" : "1970/2070" }

compaction_ioConfig = {
    "type" : "compact",
    "inputSpec" : compaction_ioConfig_inputSpec }

compaction_granularitySpec = { "segmentGranularity" : "DAY" }

compaction_spec = {
    "type": "compact",
    "dataSource": table_name,
    "ioConfig": compaction_ioConfig,
    "granularitySpec": compaction_granularitySpec
}

print(json.dumps(compaction_spec, indent=2))

Submit the task by running the next cell.

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/task", json.dumps(compaction_spec), headers=druid_headers)

Wait for a moment for the task to run.

Keep running the cell below, and you will see the table data layout change to just one segment for the entire table.

In [ ]:
sql=f'''
SELECT
  "start",
  "end",
  "size"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

### Apply a different CLUSTERED BY scheme through compaction

During the initial ingestion that you performed, notice that there was no CLUSTERED BY clause and, in this way, mimics what happens with streaming ingestion where secondary partitioning is not applied.

Use the [`partitionsSpec`](https://druid.apache.org/docs/latest/ingestion/native-batch-simple-task#partitionsspec) inside a compaction job's [`tuningConfig`](https://druid.apache.org/docs/latest/ingestion/native-batch-simple-task#tuningconfig) to apply a data clustering scheme to your table, enabling greater parallelisation and pruning of filtering operations on the dimensions you cluster by.

In the sections that follow, you will apply two techniques: hash and multi-dimension range partitioning.

#### See the table layout when hash partitioning is used

To set up hash partitioning, a `partitionSpec` needs to be added to the `tuningConfig` of the compaction job definition.

Run the next cell to create a `compaction_tuningConfig_partitionsSpec` object. Notice that it contains the configuration needed to partition the table using hashing against the `channel` dimension.

In [ ]:
compaction_tuningConfig_partitionsSpec = {
    "type" : "hashed",
    "partitionDimensions" : [
        "channel" ] }

Now define a `compaction_tuningConfig` object that will act as the `tuningConfig`.

For compaction, this:

* Sets the type to `index_parallel` - this instructs the compaction job to process the table data using native tasks.
* Enables [perfect roll-up](https://druid.apache.org/docs/latest/ingestion/rollup#perfect-rollup-vs-best-effort-rollup) - this is required when partitioning by specific dimensions.

These two objects are then incorporated into a new section, `tuningConfig`, in the compaction spec, and submitted.

In [ ]:
compaction_tuningConfig = {
    "type" : "index_parallel",
    "forceGuaranteedRollup" : "true",
    "partitionsSpec" : compaction_tuningConfig_partitionsSpec }

Run the next cell to incorporate this new section into the compaction specification.

In [ ]:
compaction_spec = {
    "type": "compact",
    "dataSource": table_name,
    "ioConfig": compaction_ioConfig,
    "tuningConfig" : compaction_tuningConfig,
    "granularitySpec": compaction_granularitySpec
}

print(json.dumps(compaction_spec, indent=2))

Run the next cell to submit the job.

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/task", json.dumps(compaction_spec), headers=druid_headers)

Run the following cell to watch as the compaction job applies a new data layout to the table.

When finished, you will see that the `shard_spec` shows the [Murmur32 hash function](murmur3_32_abs) was applied.

In [ ]:
sql=f'''
SELECT
  "start",
  "end",
  "shard_spec",
  "num_rows",
  "size"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

#### See the table layout when multi-dimension range partitioning is used

Now apply the compaction again, this time using [multi-dimension range partitioning](https://druid.apache.org/docs/latest/ingestion/native-batch/#multi-dimension-range-partitioning), which effectively creates a periodic range index across the dimensions.

Notice that the `partitionsSpec` will use `range`-type partitioning, and that multiple dimensions will be used. For the purposes of this notebook, a (for example purposes only!) target of 10000 rows per segment has been set.

In [ ]:
compaction_tuningConfig_partitionsSpec = {
    "type" : "range",
    "partitionDimensions" : [
        "isRobot", "channel" ],
    "targetRowsPerSegment" : 10000 }

compaction_tuningConfig = {
    "type" : "index_parallel",
    "forceGuaranteedRollup" : "true",
    "partitionsSpec" : compaction_tuningConfig_partitionsSpec }

compaction_spec = {
    "type": "compact",
    "dataSource": table_name,
    "ioConfig": compaction_ioConfig,
    "tuningConfig" : compaction_tuningConfig,
    "granularitySpec": compaction_granularitySpec
}

print(json.dumps(compaction_spec, indent=2))

Run the next cell to submit the compaction job.

In [ ]:
requests.post(f"{druid_host}/druid/indexer/v1/task", json.dumps(compaction_spec), headers=druid_headers)

As before, run the cell below to watch as the new layout is applied.

In [ ]:
sql=f'''
SELECT
  "start",
  "end",
  "shard_spec"
FROM sys.segments
WHERE datasource = '{table_name}'
ORDER BY 1
'''

display.sql(sql)

When the task is finished, you will see two segment files, with ranges of partition values shown in the `shard_spec` - the first runs from the start of the data ("start":null) through to isRobot:false; channel:#pl.wikipedia, the second from that same position through to the end ("end":null). Consequently, queries that compare values below / above this cut line will be parallelised across two segments.

## Clean up

Run the following cell to drop the table used in this notebook from the database.

In [ ]:
print(f"Drop datasource: [{druid.datasources.drop(table_name)}]")

## Summary

* Compaction processes existing table data.
* Compaction can run in manual or automatic mode.
* Manually apply a different primary partitioning by running a compaction task using `segmentGranularity`.
* Incorporate a `partitionsSpec` to apply partitioning on other dimensions.

## Learn more

* Read about [time-based partitioning](https://druid.apache.org/docs/latest/multi-stage-query/concepts#partitioning-by-time) in Druid.
* Also read about performance and storage [impacts of clustering](https://druid.apache.org/docs/latest/multi-stage-query/concepts#clustering) table data.
* Check out the docs on [compaction](https://druid.apache.org/docs/latest/data-management/compaction), and how to apply automatic compaction.
* Try out different partitioning methods and assess the segment layout.
* Move on to pushing your compaction specification to automatic compaction for the table.